In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000) ## 800GB?

from calitp.tables import tbl
from calitp import query_sql, magics
import calitp.magics
import branca

import shared_utils
from dla_utils import _dla_utils as dla_utils

from siuba import *
import pandas as pd

import datetime as dt
import time
from zoneinfo import ZoneInfo

import importlib

import gcsfs
fs = gcsfs.GCSFileSystem()

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

from IPython.display import display, Markdown


/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
import utils

In [3]:
#adding cal-itp id 
bbb = 300
st = 290

In [4]:
pd.set_option("display.max_columns", 100)


In [5]:
rt_sched = utils.read_data()

In [6]:
# changing calitp_deleted_at to datetime so we can filter on that 
rt_sched['calitp_deleted_at'] = pd.to_datetime(rt_sched['calitp_deleted_at'])

In [7]:
#filtering down to deleted_at to filter out the SamTrans entries from old feed
rt_sched = (rt_sched>>filter(_.calitp_deleted_at > '2022-07-01'))

In [8]:
rt_sched.head()

,calitp_itp_id,agency_name,calitp_url_number,route_id,route_short_name,service_date,calitp_extracted_at,calitp_deleted_at,num_sched,num_vp,pct_w_vp,weekday,month
0,290,SamTrans,1,110,110,2022-06-28,2021-04-16,2099-01-01,48,48,1.00,Tuesday,June
1,290,SamTrans,1,110,110,2022-06-20,2021-04-16,2099-01-01,48,19,0.40,Monday,June
2,290,SamTrans,1,110,110,2022-06-29,2021-04-16,2099-01-01,48,48,1.00,Wednesday,June
3,290,SamTrans,1,110,110,2022-06-24,2021-04-16,2099-01-01,48,28,0.58,Friday,June
4,290,SamTrans,1,110,110,2022-06-22,2021-04-16,2099-01-01,48,27,0.56,Wednesday,June


In [9]:
rt_sched.tail()

,calitp_itp_id,agency_name,calitp_url_number,route_id,route_short_name,service_date,calitp_extracted_at,calitp_deleted_at,num_sched,num_vp,pct_w_vp,weekday,month
6893,300,Big Blue Bus,0,3479,1,2022-05-27,2022-03-08,2022-09-01,180,177,0.98,Friday,May
6894,300,Big Blue Bus,0,3479,1,2022-06-10,2022-03-08,2022-09-01,180,161,0.89,Friday,June
6895,300,Big Blue Bus,0,3479,1,2022-06-30,2022-03-08,2022-09-01,180,177,0.98,Thursday,June
6896,300,Big Blue Bus,0,3479,1,2022-05-11,2022-03-08,2022-09-01,180,179,0.99,Wednesday,May
6897,300,Big Blue Bus,0,3479,1,2022-05-17,2022-03-08,2022-09-01,180,180,1.00,Tuesday,May


In [10]:
rt_sched.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3027 entries, 0 to 6897
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   calitp_itp_id        3027 non-null   int64         
 1   agency_name          2692 non-null   object        
 2   calitp_url_number    3027 non-null   int64         
 3   route_id             3027 non-null   object        
 4   route_short_name     3027 non-null   object        
 5   service_date         3027 non-null   datetime64[ns]
 6   calitp_extracted_at  3027 non-null   object        
 7   calitp_deleted_at    3027 non-null   datetime64[ns]
 8   num_sched            3027 non-null   int64         
 9   num_vp               3027 non-null   int64         
 10  pct_w_vp             3027 non-null   float64       
 11  weekday              3027 non-null   object        
 12  month                3027 non-null   object        
dtypes: datetime64[ns](2), float64(1),

In [11]:
rt_sched.agency_name.value_counts()

SamTrans        1690
Big Blue Bus    1002
Name: agency_name, dtype: int64

In [12]:
rt_sched.calitp_itp_id.value_counts()

290    2025
300    1002
Name: calitp_itp_id, dtype: int64

## Charting

In [13]:
import altair as alt

from shared_utils import altair_utils
from shared_utils import geography_utils
from shared_utils import calitp_color_palette as cp
from shared_utils import styleguide

#import seaborn as sns

In [14]:
rt_sched.sample()

,calitp_itp_id,agency_name,calitp_url_number,route_id,route_short_name,service_date,calitp_extracted_at,calitp_deleted_at,num_sched,num_vp,pct_w_vp,weekday,month
4706,290,None,0,294-199,294,2022-06-25,2022-06-19,2022-07-26,30,0,0.00,Saturday,June


In [15]:
rt_sched>>count(_.month)

,month,n
0,June,1674
1,May,1353


In [16]:
rt_sched>>group_by(_.calitp_itp_id, _.month)>>summarize(avg = _.pct_w_vp.mean())

,calitp_itp_id,month,avg
0,290,June,0.45
1,290,May,0.79
2,300,June,0.86
3,300,May,0.84


In [17]:
rt_sched>>group_by(_.agency_name, _.calitp_itp_id, _.month)>>summarize(avg = _.pct_w_vp.mean())

,agency_name,calitp_itp_id,month,avg
0,Big Blue Bus,300,June,0.86
1,Big Blue Bus,300,May,0.84
2,SamTrans,290,June,0.62
3,SamTrans,290,May,0.79


In [18]:
#june is funky

In [19]:
#aggregating by date
day_pct = utils.agg_by_date(rt_sched, 'num_sched', 'num_vp')

In [20]:
day_pct

,calitp_itp_id,agency_name,calitp_url_number,service_date,weekday,month,total_num_sched,total_num_vp,pct_w_vp
0,290,SamTrans,1,2022-05-01,Sunday,May,765,534,0.70
1,290,SamTrans,1,2022-05-02,Monday,May,1339,938,0.70
2,290,SamTrans,1,2022-05-03,Tuesday,May,1339,1284,0.96
3,290,SamTrans,1,2022-05-04,Wednesday,May,1343,3,0.00
4,290,SamTrans,1,2022-05-05,Thursday,May,1338,948,0.71
...,...,...,...,...,...,...,...,...,...
117,300,Big Blue Bus,0,2022-06-26,Sunday,June,770,498,0.65
118,300,Big Blue Bus,0,2022-06-27,Monday,June,1461,1013,0.69
119,300,Big Blue Bus,0,2022-06-28,Tuesday,June,1461,1359,0.93
120,300,Big Blue Bus,0,2022-06-29,Wednesday,June,1461,1367,0.94


In [21]:
#see if we have multiple date entries
day_pct>>count(_.service_date)>>arrange(-_.n)

,service_date,n
0,2022-05-01,2
1,2022-05-02,2
2,2022-05-03,2
3,2022-05-04,2
4,2022-05-05,2
...,...,...
56,2022-06-26,2
57,2022-06-27,2
58,2022-06-28,2
59,2022-06-29,2


## Percent by Day version 1

In [22]:
(utils.bar_chart_over_time((day_pct>>filter(_.calitp_itp_id==bbb)),
                           'service_date',
                           'pct_w_vp',
                           'pct_w_vp',
                           '%',
                           'x',
                           'Big Blue Bus Percent of Scheduled Trips with RT Vehicle Position Data')).properties(width=800)



alt.Chart(...)

In [23]:
(utils.bar_chart_over_time((day_pct>>filter(_.calitp_itp_id==st)),
                           'service_date',
                           'pct_w_vp',
                           'pct_w_vp',
                           '%',
                           'x',
                           'SamTrans Percent of Scheduled Trips with RT Vehicle Position Data')).properties(width=800)

alt.Chart(...)

In [24]:
# if we want to change the color scale
(utils.bar_chart_over_time((day_pct>>filter(_.calitp_itp_id==st)),
                           'service_date',
                           'pct_w_vp',
                           'pct_w_vp',
                           '%',
                           'x',
                           'SamTrans Percent of Scheduled Trips with RT Vehicle Position Data')).encode(color = alt.Color('pct_w_vp',
                       scale=alt.Scale(range=altair_utils.CALITP_SEQUENTIAL_COLORS),
                       legend=alt.Legend(title=(utils.labeling('pct_w_vp')),
                                         symbolLimit=10))).properties(width=800)

alt.Chart(...)

## Percent by Day Version 2

In [25]:
bar = (alt.Chart(day_pct>>filter(_.calitp_itp_id==bbb))
        .mark_bar(size=8)
        .encode(
            x=alt.X('service_date:T', title=utils.labeling('service_date'), sort=("x")),
            y=alt.Y('pct_w_vp:Q', title=utils.labeling('pct_w_vp')),
            color=alt.Color(
                'weekday',
                #scale=alt.Scale(range=altair_utils.CALITP_DIVERGING_COLORS),
                legend=alt.Legend(title=(utils.labeling('pct_w_vp'))))
        ).properties(title= "Big Blue Bus Percent of Scheduled Trips with RT Vehicle Position Data"))

# # mean line
rule = alt.Chart(day_pct>>filter(_.calitp_itp_id==bbb)
                ).mark_rule(color='red'
                           ).encode(y=alt.Y('mean(pct_w_vp):Q', axis=alt.Axis(format='%')))

(bar + rule).properties(width=800)


alt.LayerChart(...)

In [26]:
bar = (alt.Chart(day_pct>>filter(_.calitp_itp_id==st))
        .mark_bar(size=8)
        .encode(
            x=alt.X('service_date', title=utils.labeling('service_date'), sort=("x")),
            y=alt.Y('pct_w_vp', title=utils.labeling('pct_w_vp')),
            color=alt.Color(
                'weekday',
        #        scale=alt.Scale(range=altair_utils.CALITP_DIVERGING_COLORS),
                legend=alt.Legend(title=(utils.labeling('pct_w_vp'))))
        ).properties(title="SamTrans Percent of Scheduled Trips with RT Vehicle Position Data"))

# # mean line
rule = alt.Chart(day_pct>>filter(_.calitp_itp_id==bbb)).mark_rule(color='red').encode(
    y=alt.Y('mean(pct_w_vp)', axis=alt.Axis(format='%'))
)

(bar + rule).properties(width=800)

alt.LayerChart(...)

## Percent by Day Version 3

In [27]:
base = alt.Chart(day_pct).properties(width=550)
line = base.mark_trail().encode(
    x=alt.X('service_date', title=utils.labeling('service_date'), sort=("x")),
    y=alt.Y('pct_w_vp', title=utils.labeling('pct_w_vp')),
    color='agency_name',
    size='pct_w_vp:Q'
)

# mean line
rule = (alt.Chart(day_pct>>filter(_.calitp_itp_id==bbb)).mark_rule().encode(
    y=alt.Y('mean(pct_w_vp):Q', axis=alt.Axis(format='%')),
    color=alt.Color('red:N',
                    # legend=alt.Legend(values=['Average Over Time'])
                   )))

(line + rule)

alt.LayerChart(...)

## Total Number of Scheduled Trips vs Vehicle Position Trips

In [28]:
day_pct_long = day_pct.copy()
day_pct_long = day_pct_long.rename(columns={ 'total_num_sched':'Total Number Scheduled Trips',
                                            'total_num_vp':'Total Number Vehicle Position Trips'})
day_pct_long =  (day_pct_long>>select(_.agency_name,
                            _.calitp_itp_id,
                            _.service_date,
                         _['Total Number Scheduled Trips'],
                         _['Total Number Vehicle Position Trips']
                        ) 
             >> gather('measure',
                       'value',
                       _['Total Number Scheduled Trips'],
                       _['Total Number Vehicle Position Trips']
                      )
            )

In [29]:
(utils.bar_chart_over_time((day_pct_long>>filter(_.calitp_itp_id==bbb)),
                           'service_date',
                           'value', 
                           'measure',
                           ',f', 
                           'x', 
                           'Big Blus Bus Scheduled Trips with RT Vehicle Position Data')).properties(width=800)

alt.Chart(...)

In [30]:
(utils.bar_chart_over_time((day_pct_long>>filter(_.calitp_itp_id==st)),
                           'service_date',
                           'value',
                           'measure', 
                           ',f',
                           'x', 
                           'SamTrans Scheduled Trips with RT Vehicle Position Data'
                          )).properties(width=800)

alt.Chart(...)

## By Weekday

In [31]:
bbb_week = utils.groupby_onecol((day_pct>>filter(_.calitp_itp_id==bbb)), 'weekday', 'pct_w_vp')

In [32]:
bbb_week

,weekday,avg
1,Monday,0.73
5,Tuesday,0.88
6,Wednesday,0.91
4,Thursday,0.92
0,Friday,0.91
2,Saturday,0.70
3,Sunday,0.69


In [33]:
 cats_day = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

In [34]:
#using function- x axis is not sorted
utils.bar_chart_over_time(bbb_week, 
                          'weekday', 
                          'avg', 
                          'weekday', 
                          '%', 
                          cats_day,
                          'Big Blue Bus Percent of Scheduled Trips with RT Vehicle Position Data by Day').mark_bar(size=50)

alt.Chart(...)

In [35]:
bbb_week_month = utils.groupby_twocol((day_pct>>filter(_.calitp_itp_id==bbb)), 'month', 'weekday', 'pct_w_vp', 'weekday')

In [36]:
# add column arg

In [37]:
utils.bar_chart_over_time(bbb_week_month,
                          'weekday', 
                          'avg',
                          'weekday',
                          '%', 
                          cats_day,
                          'Average Big Blue Bus Percent of Scheduled Trips with RT Vehicle Position Data by Day').encode(column='month').mark_bar(size=50)

alt.Chart(...)

In [38]:
cats_month = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

In [39]:
utils.bar_chart_over_time(bbb_week_month,
                          'avg',
                          'weekday',
                          'weekday',
                          '', 
                          cats_day, 
                          'Average Big Blue Bus Percent of Scheduled Trips with RT Vehicle Position Data by Day').encode(x=alt.X('avg:Q', title=utils.labeling('avg'), axis=alt.Axis(format='%')),
                                                                                                                         row =alt.Row("month", sort=cats_month)).mark_bar(size=30)

alt.Chart(...)

In [40]:
day_pct.sample()

,calitp_itp_id,agency_name,calitp_url_number,service_date,weekday,month,total_num_sched,total_num_vp,pct_w_vp
45,290,SamTrans,1,2022-06-15,Wednesday,June,1343,733,0.55


In [41]:
(utils.bar_chart_over_time(day_pct,
                           'service_date',
                           'pct_w_vp',
                           'agency_name', 
                           '%', 
                           cats_month,
                           "Average Percent of Scheduled Trips with RT Vehicle Position Data Over Time")).mark_line(size=3).properties(width=600)

alt.Chart(...)

In [42]:
(utils.bar_chart_over_time(bbb_week_month,
                           'weekday',
                           'avg',
                           'month',
                           '%',
                           cats_day,
                           "Average Percent of Scheduled Trips with RT Vehicle Position Data by Day")).mark_line(size=3).properties(width=600)

alt.Chart(...)

### Days with percents over 95% vehicle positions data present 

In [43]:
day_pct>>filter(_.pct_w_vp>.95)

,calitp_itp_id,agency_name,calitp_url_number,service_date,weekday,month,total_num_sched,total_num_vp,pct_w_vp
2,290,SamTrans,1,2022-05-03,Tuesday,May,1339,1284,0.96
5,290,SamTrans,1,2022-05-06,Friday,May,1339,1292,0.96
11,290,SamTrans,1,2022-05-12,Thursday,May,1338,1285,0.96
12,290,SamTrans,1,2022-05-13,Friday,May,1339,1274,0.95
16,290,SamTrans,1,2022-05-17,Tuesday,May,1339,1283,0.96
17,290,SamTrans,1,2022-05-18,Wednesday,May,1343,1287,0.96
18,290,SamTrans,1,2022-05-19,Thursday,May,1338,1289,0.96
19,290,SamTrans,1,2022-05-20,Friday,May,1339,1288,0.96
26,290,SamTrans,1,2022-05-27,Friday,May,1339,1278,0.95
29,290,SamTrans,1,2022-05-30,Monday,May,765,732,0.96


## Group By Month

In [44]:
avg_month = utils.groupby_twocol(day_pct, 'agency_name', 'month', 'pct_w_vp', 'month')

In [45]:
(utils.bar_chart_over_time(avg_month, 
                           'month', 
                           'avg', 
                           'agency_name',
                           '%', 
                           cats_month, 
                           "")).encode(column='agency_name').mark_bar(size=150)

alt.Chart(...)

In [46]:
agg_weekday = utils.groupby_twocol(day_pct, 
                                   'agency_name',
                                   'weekday',
                                   'pct_w_vp', 
                                   'weekday')

In [47]:
(utils.bar_chart_over_time(agg_weekday,
                           'weekday', 
                           'avg',
                           'agency_name',
                           '%', 
                           cats_month,
                           "")).encode(column='agency_name').mark_bar(size=50)

alt.Chart(...)

## Single Operator vs. All Operator Average

* all operator average: get an average pct_w_vp for each day

In [191]:
#using utils function to get the average of the percent trips with vehicle positions per day
agg_all = (utils.groupby_onecol(rt_sched, 'service_date', 'pct_w_vp'))

In [49]:
agg_all

,service_date,avg
0,2022-05-01,0.70
1,2022-05-02,0.71
2,2022-05-03,0.93
3,2022-05-04,0.37
4,2022-05-05,0.79
...,...,...
56,2022-06-26,0.42
57,2022-06-27,0.44
58,2022-06-28,0.58
59,2022-06-29,0.59


In [143]:
from altair.expr import datum

In [149]:
def total_average_with_1op_chart(full_df, df_avg, calitp_id):
    
    avg_line = (alt.Chart(df_avg).mark_line(color='#8CBCCB').encode(x=alt.X('service_date', title=utils.labeling('service_date'), sort=("x")),
                                     y=alt.Y('avg', title=utils.labeling('avg'), axis=alt.Axis(format='%')),
                                                     color = alt.Color(title='Overall Average')
                                    ))
    
    one_op = (utils.groupby_onecol((full_df >> filter(_.calitp_itp_id == calitp_id)), 'service_date', 'pct_w_vp'))
    one_op_line = (alt.Chart(one_op).mark_line(color='#EB9F3C').encode(x=alt.X('service_date', title=utils.labeling('service_date'), sort=("x")),
                                     y=alt.Y('avg', title=utils.labeling('avg'), axis=alt.Axis(format='%'))
                                                                      ))
    chart = (avg_line + one_op_line).properties(width=550)
                              
    return chart

In [150]:
total_average_with_1op_chart(rt_sched, agg_all, 300)

alt.LayerChart(...)

In [146]:
# avg_line = (alt.Chart(df_avg).mark_line(color = 'blue').encode(x=alt.X('service_date', title=utils.labeling('service_date'), sort=("x")),
#                                      y=alt.Y('avg', title=utils.labeling('avg'), axis=alt.Axis(format='%'))
#                                     ).properties(title= 'Overall Average for Percent Trips with '))
    
# one_op_line = (alt.Chart(one_op).mark_line(color='red').encode(x=alt.X('service_date', title=utils.labeling('service_date'), sort=("x")),
#                                      y=alt.Y('avg', title=utils.labeling('avg'), axis=alt.Axis(format='%'))))
    
# return (avg_line + one_op_line).properties(width=550)

In [160]:
agg_all = (utils.groupby_onecol(rt_sched, 'service_date', 'pct_w_vp')).rename(columns={'avg':'total_average'})

In [161]:
agg_all.sample()

,service_date,total_average
15,2022-05-16,0.70


In [158]:
one_op = (utils.groupby_onecol((rt_sched >> filter(_.calitp_itp_id == 300)), 'service_date', 'pct_w_vp'))
one_op = one_op.rename(columns={'avg':'operator_average'})

# one_op_line = (alt.Chart(one_op).mark_line().encode(x=alt.X('service_date', title=utils.labeling('service_date'), sort=("x")),
#                                      y=alt.Y('avg', title=utils.labeling('avg'), axis=alt.Axis(format='%'))))

In [159]:
one_op.sample()

,service_date,operator_average
33,2022-06-03,0.84


In [167]:
by_date = pd.merge(agg_all, one_op, on= 'service_date', how='left')

In [169]:
by_date.sample()

,service_date,total_average,operator_average
19,2022-05-20,0.93,0.91


In [174]:
total = (alt.Chart(by_date).mark_line(color='#8CBCCB').encode(x=alt.X('service_date', title=utils.labeling('service_date'), sort=("x")),
                                     y=alt.Y('total_average', title=utils.labeling('avg'), axis=alt.Axis(format='%')),
                                                     color = alt.Color(title='Overall Average')
                                    ))
op = (alt.Chart(by_date).mark_line(color = '#EB9F3C').encode(x=alt.X('service_date', title=utils.labeling('service_date'), sort=("x")),
                                     y=alt.Y('operator_average', title=utils.labeling('operator_average'), axis=alt.Axis(format='%')),
                                                     color = alt.Color(title='Operator Average')
                                    ))

total+op

alt.LayerChart(...)

In [198]:
## adding legend
#https://stackoverflow.com/questions/65226756/altair-layered-line-chart-with-legend-and-custom-colors
scale = alt.Scale(domain=['total_average', 'operator_average'], range=['#8CBCCB', '#EB9F3C'])

total = alt.Chart(by_date).mark_line().transform_fold(
    fold=['total_average'], 
    as_=['variable', 'Total Average']
).encode(
    x='service_date:T', 
    y='total_average:Q', 
    color=alt.Color('variable:N', scale=scale)
)

op = alt.Chart(by_date).mark_line().transform_fold(
    fold=['operator_average'], 
    as_=['variable', 'Operator Average']
).encode(
    x= alt.X('service_date:T', title = 'Service Date'),
    y=alt.Y('operator_average:Q', title='Percent'), 
    color=alt.Color('variable:N', scale=scale)
)
(total + op).properties(width=550)

# alt.Chart(by_date).mark_line().transform_fold(
#     fold=['total_average', 'operator_average'], 
#     as_=['variable', 'value']
# ).encode(
#     x=alt.X('service_date', title=utils.labeling('service_date')),
#      y=alt.Y('value:Q', title=utils.labeling('avg')),
#     color=alt.Color('variable:N', scale=scale)
# ).properties(width=550)

alt.LayerChart(...)

#### Working through differing averages 
* no differences due to filtering out expired feeds
* needed to filter the calitp_deleted_at: `filter(_.calitp_deleted_at > '2022-07-01'))`

In [52]:
#starting with single date to work through the difference

In [53]:
# (utils.groupby_onecol((rt_sched>>filter(_.service_date=='2022-05-14')), 'calitp_itp_id', 'pct_w_vp'))

In [54]:
# date = rt_sched>>filter(_.service_date=='2022-05-14')

In [55]:
# (utils.groupby_onecol(date, 'calitp_itp_id', 'pct_w_vp')).avg.mean()

In [56]:
# (utils.groupby_onecol(date, 'calitp_itp_id', 'pct_w_vp'))>>summarize(avg = _.avg.mean())

In [57]:
# agg_all>>filter(_.service_date=='2022-05-14')

In [58]:
# date >>group_by(_.service_date)>>summarize(avg = _.pct_w_vp.mean())

In [59]:
# (rt_sched>>group_by(_.service_date)>>summarize(avg = _.pct_w_vp.mean()))>>filter(_.service_date=='2022-05-14')

In [60]:
# ((rt_sched>>group_by(_.calitp_itp_id, _.service_date)>>summarize(avg = _.pct_w_vp.mean()))>>filter(_.service_date=='2022-05-14'))>>summarize(avg2 = _.avg.mean())

In [61]:
# ((rt_sched>>group_by(_.agency_name, _.service_date)>>summarize(avg = _.pct_w_vp.mean()))>>filter(_.service_date=='2022-05-14'))>>summarize(avg2 = _.avg.mean())

In [62]:
# date>>group_by(_.agency_name)>>count(_.calitp_itp_id)

In [63]:
# date>>filter(_.agency_name.isnull())>>count(_.calitp_itp_id)

In [64]:
# date>>filter(_.calitp_itp_id==290,
#                  _.agency_name.isnull())

In [65]:
# (date>>filter(_.calitp_itp_id==290,
#                 _.calitp_url_number==1) >>arrange(_.route_id))

In [66]:
# (date>>filter(_.calitp_itp_id==300) >>arrange(_.route_id))

In [67]:
# rt_sched['calitp_deleted_at'] = pd.to_datetime(rt_sched['calitp_deleted_at'])

In [68]:
# date.calitp_deleted_at.info()

In [69]:
# (date >> filter(_.calitp_deleted_at > '2022-07-01'))>>filter(_.calitp_itp_id==290)>>count(_.calitp_url_number)

In [70]:
# # got it- filtering calitp_deleted_at to more recent
# date = (date >> filter(_.calitp_deleted_at > '2022-07-01'))

In [71]:
# (date>>group_by(_.calitp_itp_id)>>summarize(avg=_.pct_w_vp.mean()))

In [72]:
# (date>>group_by(_.calitp_itp_id)>>summarize(avg=_.pct_w_vp.mean())).avg.mean()

In [73]:
# (utils.groupby_onecol(date, 'service_date', 'pct_w_vp'))

## Using Functions (without DBT tables)

In [74]:
#gtfs_daily = utils.load_schedule_data(analysis_date_start, analysis_date_end, itp_id)

In [75]:
#gtfs_daily

In [76]:
#gtfs_daily.service_date.min()

In [77]:
#gtfs_daily.service_date.max()

In [78]:
#rt = utils.load_rt_data(analysis_date_start, analysis_date_end)

In [79]:
#rt

In [80]:
#rt['str_len'] = rt.trip_id.str.len()

In [81]:
#rt.str_len.value_counts()

In [82]:
#len(rt)

In [83]:
#rt.trip_id.nunique()

In [84]:
#len(gtfs_daily)

In [85]:
#gtfs_daily.trip_id.nunique()

In [86]:
#date_list =['2022-05-01','2022-05-02', '2022-05-31']

In [87]:

# from datetime import date, timedelta
# start_date = date(2022, 5, 1)
# end_date = date(2022, 5, 31)

In [88]:
# daterange = pd.date_range(start_date, end_date)

In [89]:
# (daterange)

In [90]:
#above range does not work with function

In [91]:
# dates = set()

# def daterange2(start, end):
#     for n in range(int((end - start).days) + 1):
#         yield start + timedelta(n)

# for single_date in daterange2(start_date, end_date):
#     dates.add(single_date.strftime('%Y-%m-%d'))

In [92]:
# (dates)

In [93]:
# rt['date'] = pd.to_datetime(rt['date'])

In [94]:
# gtfs_daily['service_date'] = pd.to_datetime(gtfs_daily['service_date'])

In [95]:
# single_date2 = '2022-05-08'

# (rt>>filter(_.date == single_date2))


In [96]:
# def get_pct_ran_df2(itp_id, list_of_dates):
#     pcts = []
#     for single_date in list_of_dates:
#         gtfs_daily2 = (gtfs_daily>>filter(_.service_date == single_date))
#         rt2 = (rt>>filter(_.date == single_date))
#         sched_rt_df = (pd.merge(gtfs_daily2, rt2, how='outer', on='trip_id', indicator='have_rt'))
#         #pct_ran = (utils.get_pct_ran(sched_rt_df, single_date))
#         day_pct_ran = {}
#         day_pct_ran['date'] = single_date
#         if ((len(sched_rt_df))!=0):
#             day_pct_ran['pct_trips_ran'] = ((len(sched_rt_df>>filter(_.have_rt=='both')))/(len(gtfs_daily2)))
#         elif ((len(sched_rt_df))==0):
#             day_pct_ran['pct_trips_ran'] = ''
#         pct_ran = pd.DataFrame([day_pct_ran])
#         pct_ran['n_have_rt'] = (len(sched_rt_df>>filter(_.have_rt=='both')))
#         pct_ran['n_missing_rt'] = (len(sched_rt_df>>filter(_.have_rt=='right_only')))
#         pct_ran['unmatched_rt'] = (len(sched_rt_df>>filter(_.have_rt=='left_only')))
#         pct_ran['nunique_sched'] = (gtfs_daily2.trip_id.nunique())
#         pct_ran['nunique_rt'] = (rt2.trip_id.nunique())

#         pcts.append(pct_ran)                                                    
#         #code help from: https://stackoverflow.com/questions/28669482/appending-pandas-dataframes-generated-in-a-for-loop
#     pcts = pd.concat(pcts)
#     pcts = pcts>>arrange(_.date)
#     return pd.DataFrame(pcts)


In [97]:
# may = utils.get_pct_ran_df(itp_id, dates, gtfs_daily, rt)

In [98]:
# may